In [25]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import pickle

import os
import sys

import warnings
warnings.filterwarnings('ignore')

src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

from d03_models.app_demo import recipe_recommendations_app

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
data_matrix = pd.read_csv('../../data/05_model_output/data_matrix_sim.csv')
data_matrix.set_index('Unnamed: 0', inplace=True)

infile = open('../../data/03_processed/crf_ingred_dict','rb')
recipe_dict = pickle.load(infile)
infile.close()

infile = open('../../data/03_processed/crf_links_dict','rb')
recipe_links_dict = pickle.load(infile)
infile.close()

infile = open('../../data/03_processed/crf_tag_dict','rb')
recipe_tags_dict = pickle.load(infile)
infile.close()

database_food_list = list(data_matrix.columns)

In [46]:
def _extract_ingredients(single_recipe_ing_list):
    ret = []
    for ingredient in single_recipe_ing_list:
        ret.append(ingredient['name'])
    return ret

def _ingredient_recipe_matcher(ingredients_list, shopping_list, recipe_title):
    matching_recipes = []
    matching_ingredients = []
    return_recipes = []

    for element in shopping_list:
        if any(element in s for s in ingredients_list):
            matching_recipes.append(recipe_title)
        count_recipe_dict = dict(Counter(matching_recipes))

    for key, val in count_recipe_dict.items():
        if val >= 2:
            return_recipes.append(key)

    if len(return_recipes) > 0:
        return return_recipes

def _find_matching_recipe(recipe_dict, shopping_list, meal, dietary_preference):
    listofkeys = []
    recipes = []
    recipe_final = []
    restricted_recipes = []
    ret = []
    listofitems = recipe_dict.items()

    for item in listofitems:
        ingred_list = _extract_ingredients(item[1])
        recipe_title = item[0]
        recipes.append(_ingredient_recipe_matcher(ingred_list, shopping_list, recipe_title))
    filtered_recipes = list(filter(None, recipes))

    for lst in filtered_recipes:
        for item in lst:
            recipe_final.append(item)

    # check if meal and dietary preferences met
    if meal is not None:
        for title in recipe_final:
            if meal in recipe_tags_dict[title]:
                restricted_recipes.append(title)
    else:
        restricted_recipes = recipe_final

    if dietary_preference is not None:
        for diet in restricted_recipes:
            if dietary_preference in recipe_tags_dict[diet]:
                ret.append(diet)
    else:
        ret = restricted_recipes

    return ret

###################################################
############## MAIN APPLICATION ###################
###################################################

def recipe_recommendations_app(shopping_basket, recipe_dict, meal, dietary_preference):

    data_matrix = pd.read_csv('../../data/05_model_output/data_matrix_sim.csv')
    data_matrix.set_index('Unnamed: 0', inplace=True)
    database_food_list = list(data_matrix.columns)

    items_in_database = []
    item_missing = []
    for item in shopping_basket:
        if item in database_food_list:
            items_in_database.append(item)
        else:
            item_missing.append(item)
#     if len(item_missing) > 0:
#         print('Items Not Found in Database: ', item_missing)
    ##############################################
    # search instacart matrix for similair items #
    ##############################################
    known_user_likes = items_in_database
    # Construct a new dataframe with the 10 closest neighbours (most similar)
    # for each artist.
    data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(1,11))
    for i in range(0, len(data_matrix.columns)):
        data_neighbours.ix[i,:10] = data_matrix.ix[0:,i].sort_values(ascending=False)[:10].index
    # Get the artists the user has played.

    # Construct the neighbourhood from the most similar items to the
    # ones our user has already liked.
    most_similar_to_likes = data_neighbours.loc[known_user_likes]
    similar_list = most_similar_to_likes.values.tolist()
    similar_list = list(set([item for sublist in similar_list for item in sublist]))

    return  _find_matching_recipe(recipe_dict, similar_list, meal, dietary_preference)

## DEMO - YOUR PERSONAL COOKBOOK

In [47]:
item1 = input("First Product: ")
item2 = input("Second Product: ")
item3 = input("Third Product: ")
item4 = input("Fourth Product: ")
item5 = input("Fith Product: ")
meal_input = input("Breakfast: Input 1 || Dinner: Input 2: ")
dietary_preference_input = input("Vegetarian: Input 1 || Omnivore: Input 2: ")

if meal_input == "1":
    meal = 'Breakfast'
else: 
    meal = 'Dinner'
    
if dietary_preference_input == "1":
    dietary_preference = 'Vegetarian'
else:
    dietary_preference = None

shopping_basket = [item1, item2, item3, item4, item5]
recipe_recommendations_app(shopping_basket, recipe_dict, meal, dietary_preference)

First Product: chili
Second Product: hot dogs
Third Product: fries
Fourth Product: cheese
Fith Product: corn
Breakfast: Input 1 || Dinner: Input 2: 2
Vegetarian: Input 1 || Omnivore: Input 2: 2


['ginger pork rice bowls',
 'hawaiian spam tacos with pineapple',
 'italian grilled cheese sandwiches',
 'classic meatloaf',
 'swedish meatballs',
 'sausage, peppers, and onions',
 'chicken panzanella salad',
 'easy chicken skillet enchiladas',
 'chicken piccata pasta',
 'skillet chicken parmesan',
 'sheet pan chicken fajitas with peppers and onions',
 'goldfish chicken tenders',
 'one-pot chicken and rice soup',
 'baked chicken taquitos',
 'slow cooker shredded chicken chili',
 'rotisserie chicken stroganoff',
 'cheesy bruschetta chicken cutlets',
 'vietnamese-style sticky chicken skewers',
 'chicken fried rice',
 'salsa verde chicken bake',
 'sheet pan chicken with roasted broccoli and potatoes',
 'baked chicken parmesan',
 'sheet pan chicken with asparagus and potatoes',
 'quick chicken pho',
 'persian pomegranate chicken (fesenjan)',
 'miso glazed chicken wings',
 'teriyaki chicken lettuce wraps',
 'sesame chicken fingers with spicy orange dipping sauce',
 'chicken peanut curry',
 